In [1]:
!gdown --id 1zu97TwzyU2T8OVvQ2VMof-XqHDX_fxPf
!gdown --id 1SjOTIPYvYK9yECH8-SDSP3b7FtqjHYzZ

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1zu97TwzyU2T8OVvQ2VMof-XqHDX_fxPf
To: /content/data_merged.csv
100% 227M/227M [00:02<00:00, 102MB/s]
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1SjOTIPYvYK9yECH8-SDSP3b7FtqjHYzZ
To: /content/analysed.csv
100% 71.1M/71.1M [00:01<00:00, 57.7MB/s]


In [2]:
import pandas as pd
import numpy as np

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from sklearn.linear_model import (
    LinearRegression, 
    SGDRegressor, 
    LogisticRegression
)
from sklearn.svm import LinearSVC

# from sklearn.preprocessing import normalize
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

SEED = 42
TEST_SIZE = 0.2

In [23]:
data = pd.read_csv('./data_merged.csv')
reviews = pd.read_csv('./analysed.csv')

In [24]:
print(data.keys())
print(reviews.keys())

Index(['Unnamed: 0', 'adult', 'belongs_to_collection', 'budget', 'genres',
       'id', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'status', 'title', 'video', 'vote_average',
       'vote_count', 'cast', 'crew', 'keywords'],
      dtype='object')
Index(['Unnamed: 0', 'Movie', 'Review', 'Results'], dtype='object')


In [5]:
def get_filtered_X(X, relevant_columns):
    non_relevant_cols = [col for col in data.columns if col not in relevant_columns]
    
    for col in non_relevant_cols:
        X = X.drop(col, axis=1)
    
    return X.to_numpy()

In [6]:
import re
import ast

for i in range(len(data['crew'])): 
  names = ""
  try:
    data['crew'][i] = data['crew'][i].replace("[","").replace("]","").replace("},","} -")
    x = data['crew'][i].split(" - ")
    for j in x:
      try:
        j = ast.literal_eval(j)
        names = names + j['name'].lower() + ":"
      except:
        pass
    data['crew'][i] = names
  except:
    data['crew'][i] = ""

for i in range(len(data['cast'])): 
  names = ""
  try:
    data['cast'][i] = data['cast'][i].replace("[","").replace("]","").replace("},","} -")
    x = data['cast'][i].split(" - ")
    for j in x:
      try:
        j = ast.literal_eval(j)
        names = names + j['name'].lower() + ":"
      except:
        pass
    data['cast'][i] = names
  except:
    data['cast'][i] = ""

for i in range(len(data['genres'])): 
  names = ""
  try:
    data['genres'][i] = data['genres'][i].replace("[","").replace("]","").replace("},","} -")
    x = data['genres'][i].split(" - ")
    for j in x:
      try:
        j = ast.literal_eval(j)
        names = names + j['name'].lower() + ":"
      except:
        pass
    data['genres'][i] = names
  except:
    data['genres'][i] = ""
    


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.p

In [29]:
movie_reviews = {}
for ind,i in enumerate(reviews['Movie']):
  movie_reviews[i.lower()] = reviews['Results'][ind] 

In [48]:
Cast = []
Actors = {}
Directors = {}
Genres = {}

Actors2 = {}
Directors2 = {}
Genres2 = {}
Crew = []
Genre = []

Reviews = []
# #weighed rating 
# C = data['vote_average'].mean()
# m = data['vote_count'].quantile(0.9)
def extract_feats():
  top_directors = ['quentin tarantino', 'christopher nolan', 'martin scorsese', 'steven spielberg']
  top_genres = ['adventure', 'science fiction', 'thriller', 'drama', 'action', 'fantasy']
  # genre_matrix = {'adventure' : 0.8, 'science fiction' : 0.55, 'thriller' : 0.6, 'drama': 0.7, 'action' : 0.5 , 'fantasy': 0.65}
  top_actors = ['leonardo dicaprio', 'tom hanks', 'tom cruise', 'jack nicholson', 'christian bale', 'brad pitt', 'al pacino']
  
  tot = len(data['cast'])
  for ind,(a,d,g, v) in enumerate(list(zip(data['cast'],data['crew'],data['genres'],data['vote_average']))):
    s = str(data['title'][ind])
    s = s.lower()

    if s in list(movie_reviews.keys()):
      Reviews.append(movie_reviews[s])
    else:
      Reviews.append(0)
    a = a.split(':')
    d = d.split(':')
    g = g.split(':')
    for j in a:
      if j not in Actors.keys():
        Actors[j] = 0
        Actors2[j] = 0  
      Actors[j] += v
      Actors2[j] += 1

    
    for j in d:
      if j not in Directors.keys():
        Directors[j] = 0
        Directors2[j] = 0  

      Directors[j] += v
      Directors2[j] += 1

    
    for j in g:
      if j not in Genres.keys():
        Genres[j] = 0  
        Genres2[j] = 0  

      Genres[j] += v
      Genres2[j] += 1



  for i in Actors.keys():
    if Actors2[i] != 0:
      Actors[i] = Actors[i]/Actors2[i]
    else:
      Actors[i] = 0

  for i in Directors.keys():
    if Directors2[i] != 0:
      Directors[i] = Directors[i]/Directors2[i]
    else:
      Directors[i] = 0

  for i in Genres.keys():
   if Genres2[i] != 0:
      Genres[i] = Genres[i]/Genres2[i]
   else:
      Genres[i] = 0

  for i in data['cast']:
    i = i.split(':')
    cast = 0
    for j in i:
      cast += Actors[j]/len(i)
    Cast.append(cast)

  for i in data['crew']:
    i = i.split(':')
    crew = 0
    for j in i:
      crew += Directors[j]/len(i)
    Crew.append(crew)

  for i in data['genres']:
    i = i.split(':')
    genre = 0
    for j in i:
      genre += Genres[j]/len(i)
    Genre.append(genre)

extract_feats()
data = data.fillna(0)
data['release_date'] = data['release_date'].apply(
    lambda x: 2022 - int(str(x)[:4])
)

Revenue = list(data['revenue'])  
Budget = list(data['budget'])
Runtime = list(data['runtime'])
Popularity = list(data['popularity'])
Release_date = list(data['release_date'])



In [63]:
print(Cast[0:100])

[6.415721120706928, 6.244613728551458, 5.991510435591308, 5.882731541892081, 5.840194770329913, 6.5276443186652955, 6.088467936443693, 5.758765461407177, 5.6329430368276885, 6.097295450170371, 5.993686465331725, 5.716304914430656, 6.222628097240219, 6.193896331458127, 5.861663746123194, 6.674362106446155, 6.162129226274916, 6.024877345296905, 5.88688067702608, 5.781676483378675, 5.970715989280489, 6.0432954805056776, 5.864012726968815, 5.846221311369903, 5.959704355047743, 6.255004399235418, 5.989508389897868, 6.4342004175277205, 6.586921282238336, 6.224688982166492, 6.072300990864213, 6.676707440224041, 6.008361814624775, 5.891966473992257, 6.106976790154796, 6.386572893794507, 4.836837515155508, 5.756303055870714, 5.936727898982463, 6.130551544055439, 6.1992927896755115, 6.027958709212359, 5.939996301234487, 5.569780289359294, 6.15202131550353, 6.06176505576267, 6.67235123707004, 6.092277172859678, 5.5402838509427585, 6.513856522446879, 5.621011576093241, 6.033157625416457, 6.4805224

In [60]:
#normalize

r_mean = np.mean(np.array(Revenue))
r_max = np.max(np.array(Revenue))

for i in range(len(Revenue)):
  Revenue[i] = (Revenue[i] - r_mean)/r_max


r_mean = np.mean(np.array(Budget))
r_max = np.max(np.array(Budget))

for i in range(len(Budget)):
  Budget[i] = (Budget[i] - r_mean)/r_max


In [61]:
X = np.array(list(zip(Revenue,Budget,Genre,Cast,Crew, Runtime,Popularity,Reviews)))
y = np.array(list(data['vote_average']))
print(y.shape)

(46628,)


In [62]:
def process(X, y):
    new_X, new_y = [], []
    
    for Xi, yi in zip(X, y):
        if Xi[0] <= 0 or Xi[1] <= 0: continue

        new_X.append(Xi)
        new_y.append(yi)

    return new_X, new_y

X, y = process(X, y)
y = np.array(y)
print(len(X), len(y))

3399 3399


In [12]:

# rb = [float(i/j) for i,j,k,l,m,n,o,p in X]
# Genre = [k for i,j,k,l,m,n,o,p in X]
# Cast = [l for i,j,k,l,m,n,o,p in X]
# Crew = [m for i,j,k,l,m,n,o,p in X]
# Runtime = [n for i,j,k,l,m,n,o,p in X]
# Popularity = [o for i,j,k,l,m,n,o,p in X]
# Release_date = [p for i,j,k,l,m,n,o,p in X]

# X = np.array(list(zip(rb,Genre,Cast,Crew, Runtime,Popularity,Release_date,)))

# print(X.shape)


In [64]:
def regression(X, y, choice='lr'):
    X_train, X_test, y_train, y_test = train_test_split(
       X, y, test_size=TEST_SIZE, random_state=SEED
    )
    regressor = None
    if choice == 'lr':
        regressor = LinearRegression()
    elif choice == 'sgd':
        regressor = SGDRegressor(max_iter=1000, tol=1e-3)
    else:
        raise NotImplementedError

    model = make_pipeline(StandardScaler(), regressor)
    model.fit(X_train, y_train)
    print(f"{choice} score: {model.score(X_test, y_test)}")
    return

In [65]:
regression(X, y, 'lr')
regression(X, y, 'sgd')

lr score: 0.9278686622657609
sgd score: 0.92793346530979


In [ ]:
def convert_to_categorical(y):
    y = list(map(int, y))
    y = to_categorical(y)
    return y

def classification(X, y, choice='lr'):
    y = list(map(int, y))

    X_train, X_test, y_train, y_test = train_test_split(
       X, y, test_size=TEST_SIZE, random_state=SEED
    )

    classifier = None
    if choice == 'lr':
        classifier = LogisticRegression(max_iter=5000)
    elif choice == 'svc':
        classifier = LinearSVC(max_iter=5000)
    else:
        raise NotImplementedError

    model = make_pipeline(StandardScaler(), classifier)
    model.fit(X_train, y_train)
    print(f"{choice} score: {model.score(X_test, y_test)}")
    return model

In [ ]:
lr = classification(X, y, 'lr')
svc = classification(X, y, 'svc')

In [ ]:
from joblib import dump

dump(lr, 'logreg.joblib')
dump(svc, 'svc.joblib')

['svc.joblib']